In [2]:

# Preparing Data
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error,mean_squared_log_error, roc_auc_score, accuracy_score, f1_score, precision_recall_curve, log_loss
# Load the rock mines dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data"
df = pd.read_csv(url)
column_names = ["sensor_" + str(i) for i in range(1, 61)] + ["target"]
df.columns = column_names
df = df.reset_index()
df = df.rename(columns={"index": "id"})
df['target'] = df['target'].map({'M': 1, 'R': 0})
x_data = df.iloc[:, :3]
x_data = x_data.drop('id', axis=1)
y_data = df['target']
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.4, stratify=y_data)
##### Get Base Scoring
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import*
kfold = RepeatedStratifiedKFold(n_splits=10,n_repeats=5, random_state=None)
model = RandomForestClassifier()
scoring = 'accuracy'
results = cross_val_score(model, x_test, y_test, cv=kfold, scoring=scoring)
print(f'Scores: {results.mean():.2f} (+/- {results.std() * 2:.2f})')
from itertools import combinations
columns = x_data.columns
combs = []
## or with subsets with only 2-3 columns
for i in range(1, 2):
    combs += list(combinations(columns, i))
### Get Single Column Scoring
cluster_number = 5 
data_cluster_train = pd.DataFrame()
data_cluster_test = pd.DataFrame()
i = 1
for i, comb in enumerate(combs, start=1):
    print(f"Processing combination {i}/{len(combs)}", end="\r")
    kmeans = KMeans(n_clusters=cluster_number, n_init=10)
    kmeans.fit(x_train)
    data_cluster_train[comb[0]] = kmeans.labels_
    data_cluster_test[comb[0]] = kmeans.predict(x_test) 
# Setting Cluster Data
data_cluster_train.columns += '_cluster'
data_cluster_test.columns += '_cluster'
data_cluster_test = data_cluster_test.set_index(x_test.index)
data_cluster_train = data_cluster_train.set_index(x_train.index)
# Add Cluster Features to Dataframe
x_train_enc_cluster = pd.merge(data_cluster_train, x_train, left_index=True, right_index=True)
x_test_enc_cluster = pd.merge(data_cluster_test, x_test, left_index=True, right_index=True)
## Re-Evaluation
kfold = StratifiedKFold(n_splits=10, random_state=None)
model = RandomForestClassifier()
scoring = 'accuracy'
results = cross_val_score(model, x_test_enc_cluster, y_test, cv=kfold, scoring=scoring)
print(f'Scores: {results.mean():.2f} (+/- {results.std() * 2:.2f})')


Scores: 0.51 (+/- 0.36)
Scores: 0.50 (+/- 0.22)2/2
